## Instalacja

In [1]:
!pip install redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 1.7 MB/s eta 0:00:000:00:01m eta 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
import redis
import time

In [3]:
conn = redis.Redis(host='localhost', port=6379, decode_responses=True)

## Dodawanie użytkowników i przedmiotów

In [23]:
def add_user(conn, user_id,name, funds):
    user = f'users:{user_id}'
    conn.hset(user, 'name', name)
    conn.hset(user, 'funds', funds)
    return True

def add_item(conn, user_id, item):
    if conn.hgetall(f'users:{user_id}'):
        inventory = f'inventory:{user_id}'
        conn.sadd(inventory, item)
        return True
    return False
    

## Wystawianie przedmiotów na sprzedaż

In [49]:
def list_item(conn, item_id, seller_id, price):
    inventory = f'inventory:{seller_id}'
    item = f'{item_id}.{seller_id}'
    end = time.time() + 5
    pipe = conn.pipeline()

    while time.time() < end:
        try:
            pipe.watch(inventory)
            if not pipe.sismember(inventory, item_id):
                pipe.unwatch()
                return None
            pipe.multi()
            pipe.zadd("market:", {item: price})
            pipe.srem(inventory, item_id)
            pipe.execute()
            return True
        except redis.exceptions.WatchError:
            print("watch failed")
    return False
    

## Sprzedaż

In [94]:
def purchase_item(conn, buyer_id, item, seller_id, lprice):
    buyer = f'users:{buyer_id}'
    seller = f'users:{seller_id}'
    item = f'{item}.{seller_id}'
    inventory = f'inventory:{buyer_id}'
    end = time.time() + 10
    pipe = conn.pipeline()

    while time.time() < end:
        try:
            pipe.watch("market:", buyer)

            price = pipe.zscore('market:', item)
            funds = int(pipe.hget(buyer, 'funds'))
            if price != lprice or price > funds:
                pipe.unwatch()
                return False
            pipe.multi
            pipe.hincrby(seller, 'funds', int(price))
            pipe.hincrby(buyer, 'funds', int(-price))
            pipe.sadd(inventory, item)
            pipe.zrem("market:", item)
            pipe.execute()
            return True
        except redis.exceptions.WatchError:
            print("watch failed")
    return True


In [86]:
add_user(conn, 1, "Piotr", 300)
add_user(conn, 2, "Grześ", 400)
add_user(conn, 3, "Ania", 500)

True

In [87]:
conn.hgetall('users:3')

{'name': 'Ania', 'funds': '500'}

In [88]:
add_item(conn, 1, 'książkaCPP')
add_item(conn, 2, 'książkaJS')
add_item(conn, 3, 'książkaRust')

True

In [89]:
list_item(conn, 'książkaRust', 3, 500)
list_item(conn, 'książkaCPP', 1, 450)


True

In [90]:
conn.zrange('market:', 0, -1, withscores=True)

[('książkaCPP.1', 450.0), ('książkaRust.3', 500.0)]

In [96]:
purchase_item(conn, 3, 'książkaCPP', 1, 450)

False

In [104]:
conn.hgetall('users:3')

{'name': 'Ania', 'funds': '50'}

In [105]:
conn.hgetall('users:1')

{'name': 'Piotr', 'funds': '750'}

In [98]:
conn.smembers('inventory:3')

{'książkaCPP.1'}

## Zadanie

Napisać funkcję `unlist_item()`, która wraca przedmiot spowrotem do ekwipunku użytkownika

In [102]:
def unlist_item(conn, user_id, item):
    raise NotImplementedError

In [103]:
unlist_item(conn, 1,4)

NotImplementedError: 

In [107]:
unlist_item(conn, 3, "książkaRust")

True

In [108]:
conn.zrange('market:', 0, -1, withscores=True)

[]

In [109]:
conn.smembers('inventory:3')

{'książkaCPP.1', 'książkaRust'}